# Análise Exploratória - Dados Socioeconômicos do IBGE

Este notebook demonstra a extração e análise de dados socioeconômicos brasileiros utilizando a API do IBGE.

## Objetivos
1. Extrair dados de localidades (regiões, estados, municípios)
2. Analisar dados populacionais
3. Visualizar indicadores econômicos (PIB)
4. Gerar insights sobre o panorama socioeconômico brasileiro

## Fontes de Dados
- **API de Localidades**: https://servicodados.ibge.gov.br/api/docs/localidades
- **API de Agregados (SIDRA)**: https://servicodados.ibge.gov.br/api/docs/agregados

## 1. Configuração do Ambiente

In [ ]:
# Imports necessários
import sys
from pathlib import Path

# Adicionar diretório raiz ao path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Bibliotecas de análise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Ambiente configurado com sucesso!")

In [ ]:
# Importar módulos do projeto
from src.extractors import IBGEClient, LocalidadesExtractor, PopulacaoExtractor, PIBExtractor
from src.transformers import DataCleaner
from src.utils.logger import setup_logger

# Configurar logging
setup_logger(log_level="INFO")

print("Módulos do projeto carregados!")

## 2. Extração de Localidades

In [ ]:
# Criar cliente IBGE
client = IBGEClient()

# Extrair localidades
with LocalidadesExtractor(client) as extractor:
    df_regioes = extractor.extract_regioes()
    df_estados = extractor.extract_estados()
    df_municipios = extractor.extract_municipios()

print(f"Regiões extraídas: {len(df_regioes)}")
print(f"Estados extraídos: {len(df_estados)}")
print(f"Municípios extraídos: {len(df_municipios)}")

### 2.1 Regiões do Brasil

In [ ]:
# Visualizar regiões
df_regioes

### 2.2 Estados por Região

In [ ]:
# Contagem de estados por região
estados_por_regiao = df_estados.groupby('regiao_nome').size().reset_index(name='quantidade')
estados_por_regiao = estados_por_regiao.sort_values('quantidade', ascending=False)

# Visualização
fig = px.bar(
    estados_por_regiao,
    x='regiao_nome',
    y='quantidade',
    title='Quantidade de Estados por Região',
    labels={'regiao_nome': 'Região', 'quantidade': 'Número de Estados'},
    color='quantidade',
    color_continuous_scale='Blues'
)
fig.update_layout(showlegend=False)
fig.show()

### 2.3 Municípios por Estado

In [ ]:
# Contagem de municípios por estado
municipios_por_estado = df_municipios.groupby(['uf_sigla', 'uf_nome', 'regiao_nome']).size().reset_index(name='quantidade')
municipios_por_estado = municipios_por_estado.sort_values('quantidade', ascending=False)

# Top 10 estados com mais municípios
top10_municipios = municipios_por_estado.head(10)

fig = px.bar(
    top10_municipios,
    x='uf_sigla',
    y='quantidade',
    color='regiao_nome',
    title='Top 10 Estados com Mais Municípios',
    labels={'uf_sigla': 'Estado', 'quantidade': 'Número de Municípios', 'regiao_nome': 'Região'},
    text='quantidade'
)
fig.update_traces(textposition='outside')
fig.show()

In [ ]:
# Municípios por região (Treemap)
fig = px.treemap(
    municipios_por_estado,
    path=['regiao_nome', 'uf_nome'],
    values='quantidade',
    title='Distribuição de Municípios por Região e Estado',
    color='quantidade',
    color_continuous_scale='Viridis'
)
fig.show()

## 3. Dados Populacionais

In [ ]:
# Extrair dados de população
with PopulacaoExtractor(client) as extractor:
    # População do Brasil
    df_pop_brasil = extractor.extract_brasil(anos="2010|2015|2020|2021|2022|2023")
    
    # População por estado
    df_pop_estados = extractor.extract_por_estado(anos="2023")

print(f"Registros população Brasil: {len(df_pop_brasil)}")
print(f"Registros população Estados: {len(df_pop_estados)}")

### 3.1 Evolução da População Brasileira

In [ ]:
# Limpar e preparar dados
cleaner = DataCleaner()
df_pop_brasil_clean = cleaner.clean_populacao(df_pop_brasil)

# Agrupar por ano
pop_por_ano = df_pop_brasil_clean.groupby('ano')['valor'].sum().reset_index()
pop_por_ano.columns = ['ano', 'populacao']
pop_por_ano['populacao_milhoes'] = pop_por_ano['populacao'] / 1_000_000

# Gráfico de linha
fig = px.line(
    pop_por_ano,
    x='ano',
    y='populacao_milhoes',
    title='Evolução da População Brasileira',
    labels={'ano': 'Ano', 'populacao_milhoes': 'População (milhões)'},
    markers=True
)
fig.update_traces(line=dict(width=3, color='#2E86AB'))
fig.update_layout(
    yaxis_tickformat='.1f',
    hovermode='x unified'
)
fig.show()

print(pop_por_ano)

### 3.2 População por Estado (2023)

In [ ]:
# Preparar dados
df_pop_estados_clean = cleaner.clean_populacao(df_pop_estados)

# Ordenar por população
df_pop_estados_sorted = df_pop_estados_clean.sort_values('valor', ascending=True)

# Gráfico de barras horizontais
fig = px.bar(
    df_pop_estados_sorted,
    x='valor',
    y='localidade_nome',
    orientation='h',
    title='População Estimada por Estado (2023)',
    labels={'valor': 'População', 'localidade_nome': 'Estado'},
    color='valor',
    color_continuous_scale='Plasma'
)
fig.update_layout(
    height=800,
    xaxis_tickformat=',',
    showlegend=False
)
fig.show()

In [ ]:
# Top 10 estados mais populosos
top10_pop = df_pop_estados_sorted.tail(10).copy()
top10_pop['populacao_milhoes'] = top10_pop['valor'] / 1_000_000

fig = px.pie(
    top10_pop,
    values='populacao_milhoes',
    names='localidade_nome',
    title='Top 10 Estados Mais Populosos (2023)',
    hole=0.4
)
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

## 4. Dados de PIB

In [ ]:
# Extrair dados de PIB
with PIBExtractor(client) as extractor:
    # PIB por estado
    df_pib_estados = extractor.extract_pib_por_estado(anos="2019|2020|2021")
    
    # PIB per capita
    df_pib_percapita = extractor.extract_pib_percapita_por_estado(anos="2021")

print(f"Registros PIB: {len(df_pib_estados)}")
print(f"Registros PIB per capita: {len(df_pib_percapita)}")

### 4.1 PIB por Estado

In [ ]:
# Limpar dados
df_pib_clean = cleaner.clean_pib(df_pib_estados)

# Filtrar último ano disponível
ultimo_ano = df_pib_clean['ano'].max()
df_pib_ultimo = df_pib_clean[df_pib_clean['ano'] == ultimo_ano].copy()

# Converter para bilhões
df_pib_ultimo['pib_bilhoes'] = df_pib_ultimo['valor'] / 1_000_000

# Ordenar
df_pib_ultimo = df_pib_ultimo.sort_values('pib_bilhoes', ascending=False)

print(f"PIB por Estado ({ultimo_ano})")
df_pib_ultimo[['localidade_nome', 'pib_bilhoes']].head(10)

In [ ]:
# Gráfico de barras do PIB
fig = px.bar(
    df_pib_ultimo.head(15),
    x='localidade_nome',
    y='pib_bilhoes',
    title=f'PIB por Estado ({ultimo_ano}) - Top 15',
    labels={'localidade_nome': 'Estado', 'pib_bilhoes': 'PIB (R$ bilhões)'},
    color='pib_bilhoes',
    color_continuous_scale='Greens'
)
fig.update_layout(
    xaxis_tickangle=-45,
    showlegend=False
)
fig.show()

### 4.2 PIB per Capita

In [ ]:
# Limpar dados de PIB per capita
df_pibpc_clean = cleaner.clean_pib(df_pib_percapita)

# Ordenar por valor
df_pibpc_sorted = df_pibpc_clean.sort_values('valor', ascending=True)

# Gráfico de barras horizontais
fig = px.bar(
    df_pibpc_sorted,
    x='valor',
    y='localidade_nome',
    orientation='h',
    title='PIB per Capita por Estado (2021)',
    labels={'valor': 'PIB per capita (R$)', 'localidade_nome': 'Estado'},
    color='valor',
    color_continuous_scale='RdYlGn'
)
fig.update_layout(
    height=800,
    xaxis_tickformat=',.0f',
    showlegend=False
)
fig.show()

## 5. Análise Comparativa: Piauí vs Média Nacional

In [ ]:
# Filtrar dados do Piauí
piaui_pop = df_pop_estados_clean[df_pop_estados_clean['localidade_nome'].str.contains('Piauí')]
piaui_pibpc = df_pibpc_clean[df_pibpc_clean['localidade_nome'].str.contains('Piauí')]

# Médias nacionais
media_pop = df_pop_estados_clean['valor'].mean()
media_pibpc = df_pibpc_clean['valor'].mean()

print("=" * 50)
print("ANÁLISE: PIAUÍ vs MÉDIA NACIONAL")
print("=" * 50)

if not piaui_pop.empty:
    pop_pi = piaui_pop['valor'].values[0]
    print(f"\nPopulação do Piauí: {pop_pi:,.0f}")
    print(f"Média Nacional: {media_pop:,.0f}")
    print(f"Diferença: {((pop_pi/media_pop)-1)*100:.1f}% em relação à média")

if not piaui_pibpc.empty:
    pibpc_pi = piaui_pibpc['valor'].values[0]
    print(f"\nPIB per capita do Piauí: R$ {pibpc_pi:,.2f}")
    print(f"Média Nacional: R$ {media_pibpc:,.2f}")
    print(f"Diferença: {((pibpc_pi/media_pibpc)-1)*100:.1f}% em relação à média")

## 6. Exportação dos Dados

In [ ]:
# Criar diretório de saída
output_dir = project_root / 'data' / 'processed'
output_dir.mkdir(parents=True, exist_ok=True)

# Exportar dados processados
df_regioes.to_csv(output_dir / 'regioes.csv', index=False, encoding='utf-8-sig')
df_estados.to_csv(output_dir / 'estados.csv', index=False, encoding='utf-8-sig')
df_municipios.to_csv(output_dir / 'municipios.csv', index=False, encoding='utf-8-sig')

if not df_pop_estados_clean.empty:
    df_pop_estados_clean.to_csv(output_dir / 'populacao_estados.csv', index=False, encoding='utf-8-sig')

if not df_pibpc_clean.empty:
    df_pibpc_clean.to_csv(output_dir / 'pib_percapita_estados.csv', index=False, encoding='utf-8-sig')

print("Dados exportados com sucesso!")
print(f"Diretório: {output_dir}")

# Listar arquivos gerados
for f in output_dir.glob('*.csv'):
    print(f"  - {f.name}")

## 7. Resumo e Insights

### Principais Descobertas:

1. **Distribuição Geográfica**:
   - O Brasil possui 5 regiões, 27 estados e aproximadamente 5.570 municípios
   - A região Nordeste possui o maior número de estados
   - Minas Gerais é o estado com mais municípios

2. **Demografia**:
   - São Paulo é o estado mais populoso, concentrando cerca de 22% da população
   - A região Sudeste concentra a maior parte da população brasileira

3. **Economia**:
   - São Paulo responde por aproximadamente 32% do PIB nacional
   - O Distrito Federal possui o maior PIB per capita
   - Estados do Norte e Nordeste apresentam PIB per capita abaixo da média nacional

### Próximos Passos:
- Análise de séries temporais da evolução populacional
- Correlação entre indicadores socioeconômicos
- Dashboard interativo com dados atualizados

In [ ]:
# Fechar conexão
client.close()
print("Análise concluída!")